In [41]:
import pulp
import numpy as np
from contextlib import contextmanager, suppress
import contextlib

In [60]:
def pulp_optimization(A, B, verbose=True):
    prob = pulp.LpProblem("Optimization Problem", pulp.LpMinimize)

    m = A.shape[0]
    n = A.shape[1]

    max_regret_a = np.max(A) - np.min(A)
    max_regret_b = np.max(B) - np.min(B)

    # VARIABLES

    xa, xb = [], [] # strategies
    ra, rb = [], [] # regrets
    sa, sb = [], [] # supports

    def init(i, x, r, s, letter, r_bound):
        x.append(pulp.LpVariable(f"x{letter}_{i}", lowBound=0, upBound=1)) # [0, 1]
        r.append(pulp.LpVariable(f"r{letter}_{i}", lowBound=0, upBound=r_bound)) # >= 0
        s.append(pulp.LpVariable(f"s{letter}_{i}", cat=pulp.LpBinary)) # {0, 1}

    for i in range(m) :
        init(i, xa,ra,sa, "a", max_regret_a)
    for j in range(n):
        init(j, xb,rb,sb, "b", max_regret_b)

    gain_a = pulp.LpVariable("gain_a", lowBound=0)
    gain_b = pulp.LpVariable("gain_b", lowBound=0)

    # for i in range(m):
    #     prob += gain_a == pulp.lpSum(xa[i] * A[i, j] * xb[j] for j in range(n))

    # for j in range(n):
    #     prob += gain_b == pulp.lpSum(xb[j] * B[i, j] * xa[i] for i in range(n))

    # CONSTRAINTS

    # it is necessary to add delta_i

    # ---------------------------

    prob += pulp.lpSum(ra) + pulp.lpSum(rb)

    with contextlib.redirect_stdout(None):
        prob.solve()
    
    if verbose :
        print(f"Status: {pulp.LpStatus[prob.status]}")
        for v in prob.variables():
            print(f"{v.name}: {v.value()}")
        print(f"Objective: {pulp.value(prob.objective)}")
    
    solution = pulp.value(prob.objective)

    return solution

In [61]:
def nash_equilibrium_test(matrix_A, matrix_B, wanted_equilibrium):
    solution = pulp_optimization(matrix_A, matrix_B)
    print("---------------")
    ##assert solution == wanted_equilibrium, f"Error: wrong nash equilibrium. Wanted: {wanted_equilibrium}, Computed: {solution}"

In [62]:
print("Start tests ...")
print("---------------")

nash_equilibrium_test(
    np.array([[3, 2], [1, 4]]), 
    np.array([[2, 1], [3, 2]]), 
    3)

nash_equilibrium_test(
    np.array([[1, 2], [3, 4]]), 
    np.array([[4, 3], [2, 1]]), 
    2)

print("Nash equilibrium tests succeed.")

Start tests ...
---------------
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/astragaliton/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3c93b4ee8d694ffa8698f3807bdc23ab-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/3c93b4ee8d694ffa8698f3807bdc23ab-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 10 RHS
At line 11 BOUNDS
At line 16 ENDATA
Problem MODEL has 0 rows, 4 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 4 columns and 0 elements
Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
ra_0: 0.0
ra_1: 0.0
rb_0: 0.0
rb_1: 0.0
Objective: 0.0
---------------
Welcome to the CBC MILP Solver 
Ver

In [24]:

print("hi")
%%capture

print("what ?")